# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will use your trained model to generate captions for images in the test dataset.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Get Data Loader for Test Dataset 
- [Step 2](#step2): Load Trained Models
- [Step 3](#step3): Finish the Sampler
- [Step 4](#step4): Clean up Captions
- [Step 5](#step5): Generate Predictions!

<a id='step1'></a>
## Step 1: Get Data Loader for Test Dataset

Before running the code cell below, define the transform in `transform_test` that you would like to use to pre-process the test images.  

Make sure that the transform that you define here agrees with the transform that you used to pre-process the training images (in **2_Training.ipynb**).  For instance, if you normalized the training images, you should also apply the same normalization procedure to the test images.

In [1]:
import sys
sys.path.append('/mnt/ssd2/kcheng/gpu205/cocoapi/PythonAPI')
from pycocotools.coco import COCO
# from data_loader import get_loader
from torchvision import transforms
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# TODO #1: Define a transform to pre-process the testing images.
transform_test = transforms.Compose([ 
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])


# # Create the data loader.
# data_loader = get_loader(transform=transform_test,    
#                          mode='test')

## Load vocabulary wrapper
with open('./vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
print('Vocabulary successfully loaded from vocab.pkl file!')

Vocabulary successfully loaded from vocab.pkl file!


<a id='step2'></a>
## Step 2: Load Trained Models

In the next code cell we define a `device` that you will use move PyTorch tensors to GPU (if CUDA is available).  Run this code cell before continuing.

In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Before running the code cell below, complete the following tasks.

### Task #1

In the next code cell, you will load the trained encoder and decoder from the previous notebook (**2_Training.ipynb**).  To accomplish this, you must specify the names of the saved encoder and decoder files in the `models/` folder (e.g., these names should be `encoder-5.pkl` and `decoder-5.pkl`, if you trained the model for 5 epochs and saved the weights after each epoch).  

### Task #2

Plug in both the embedding size and the size of the hidden layer of the decoder corresponding to the selected pickle file in `decoder_file`.

In [3]:
# Watch for any changes in model.py, and re-load it automatically.
# % load_ext autoreload
# % autoreload 2

import os
import torch
from model import EncoderCNN, DecoderRNN

# TODO #2: Specify the saved models to load.
encoder_file = 'encoder-7-12941.pkl'
decoder_file = 'decoder-7-12941.pkl'

# TODO #3: Select appropriate values for the Python variables below.
embed_size = 256
hidden_size = 512

# The size of the vocabulary.
vocab_size = len(vocab)

# Initialize the encoder and decoder, and set each to inference mode.
encoder = EncoderCNN(embed_size)
encoder.eval()    # evaluation mode (BN uses moving mean/variance)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)
# decoder.eval()

# Load the trained weights.
encoder.load_state_dict(torch.load(os.path.join('./models-attention', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models-attention', decoder_file)))

# Move models to GPU if CUDA is available.
encoder.to(device)
decoder.to(device)

/mnt/ssd2/kcheng/miniconda3/envs/cv-nd/lib/python3.6/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


DecoderRNN(
  (embed): Embedding(8852, 256)
  (lstm): LSTM(256, 512, batch_first=True, dropout=0.4)
  (linear): Linear(in_features=512, out_features=8852, bias=True)
  (attention): Linear(in_features=768, out_features=2048, bias=True)
  (attended): Linear(in_features=2304, out_features=256, bias=True)
  (softmax): Softmax(dim=None)
)

<a id='step3'></a>
## Step 3: Finish the Sampler

Before executing the next code cell, you must write the `sample` method in the `DecoderRNN` class in **model.py**.  This method should accept as input a PyTorch tensor `features` containing the embedded input features corresponding to a single image.

It should return as output a Python list `output`, indicating the predicted sentence.  `output[i]` is a nonnegative integer that identifies the predicted `i`-th token in the sentence.  The correspondence between integers and tokens can be explored by examining either `data_loader.dataset.vocab.word2idx` (or `data_loader.dataset.vocab.idx2word`).

After implementing the `sample` method, run the code cell below.  If the cell returns an assertion error, then please follow the instructions to modify your code before proceeding.  Do **not** modify the code in the cell below. 

In [ ]:
from PIL import Image
import cv2
import json
from tqdm import tqdm

def load_image(im_path, transform=None):
    image = Image.open(im_path).convert("RGB")
    image = image.resize([256, 256], Image.LANCZOS)
    
    if transform is not None:
        image = transform(image).unsqueeze(0)  # convert [3, 224, 224] -> [1, 3, 224, 224], where 1 is batch_size
        
    return image


def clean_sentence(output):
    sentence = ''
    for x in output:
        word = vocab.idx2word[x]
        if word == '<end>':
            break
        sentence += ' ' + word
    
    sentence = sentence.strip()[len('<start>')+1:]
    if sentence.endswith('.'):
        sentence = sentence[:-2]
    
    return sentence



## Prepare Image
im_root = '/mnt/ssd2/kcheng/gpu205/cocoapi/images/val2014'

cnt = 0
sample_images = []
jsontxt = []

for imname in os.listdir(im_root):
    if '.jpg' not in imname:
        continue
    im_path = os.path.join(im_root, imname)
    img = Image.open(im_path).convert("RGB")
    img = np.asarray(img)
#     sample_images.append(img)
    
    im = load_image(im_path, transform_test)

    
    # Move image Pytorch Tensor to GPU if CUDA is available.
    image_tensor = im.to(device)

    
    # Generate caption from image.
    try:
        feature, cnn_features = encoder(image_tensor)

        # Pass the embedded image features through the model to get a predicted caption.
        output = decoder.sample(feature, cnn_features)
        output = output.cpu().data.numpy()
    except:
        continue    
    
#     assert (type(output)==list), "Output needs to be a Python list" 
#     assert all([type(x)==int for x in output]), "Output should be a list of integers." 
    assert all([x in vocab.idx2word for x in output]), "Each entry in the output needs to correspond to an integer that indicates a token in the vocabulary."
    
    # Decode word_ids to words
    sentence = clean_sentence(output)
    
    
    image_id = int(imname.strip().split('_')[2][:-4])
    jsontxt.append({"image_id":image_id, "caption":sentence})
    # Print out image and generated caption.
    print(imname + '\t' + sentence)


# generate res json
# jsondata = json.dumps(jsontxt,indent=4,separators=(',', ': '))
# f = open('./val2014_res_test.json', 'w')
# f.write(jsondata)
# f.close()


# f, (ax1, ax2) = plt.subplots(1, 2)
# f.tight_layout()
# ax1.imshow(sample_images[0])
# ax2.imshow(sample_images[1])
# plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
# plt.show()


/mnt/ssd2/kcheng/miniconda3/envs/cv-nd/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/mnt/ssd2/kcheng/gpu205/Image-Captioning-Project/model.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention_weights = self.softmax(attention_weights)


COCO_val2014_000000092363.jpg	a man riding a horse down a street
COCO_val2014_000000380348.jpg	a dining room table with a christmas tree in the middle
COCO_val2014_000000227772.jpg	a man riding a skateboard down a cement ramp
COCO_val2014_000000136734.jpg	a man is doing a trick on a skateboard
COCO_val2014_000000150026.jpg	a hot dog with a pickle and a side dish
COCO_val2014_000000293751.jpg	a herd of elephants walking across a grass field
COCO_val2014_000000016318.jpg	a man riding a skateboard down the side of a cement planter
COCO_val2014_000000062819.jpg	a man riding a surfboard on a wave in the ocean
COCO_val2014_000000308793.jpg	a group of people riding skis down a snow covered slope
COCO_val2014_000000118679.jpg	a woman holding a tennis racket on a tennis court
COCO_val2014_000000329373.jpg	a group of people standing around a living room
COCO_val2014_000000528811.jpg	a yellow and blue train traveling down tracks next to a loading platform
COCO_val2014_000000472540.jpg	a large tru

COCO_val2014_000000418440.jpg	a sandwich with a pickle and a side salad
COCO_val2014_000000468121.jpg	a group of people standing in a room
COCO_val2014_000000074030.jpg	a man standing next to a red and white horse
COCO_val2014_000000191705.jpg	a bathroom with a toilet and a urinal
COCO_val2014_000000006874.jpg	a group of giraffes standing in a field
COCO_val2014_000000206560.jpg	a man flying through the air while riding a snowboard
COCO_val2014_000000419867.jpg	a store with a bunch of bananas and a sign
COCO_val2014_000000221281.jpg	a giraffe and a zebra in a grassy area
COCO_val2014_000000101270.jpg	a group of planes flying in the air
COCO_val2014_000000220704.jpg	a kitchen with a stove , oven , sink and a counter
COCO_val2014_000000117098.jpg	a red motorcycle parked in a parking lot
COCO_val2014_000000436992.jpg	a group of people sitting on a bench in the middle of a river
COCO_val2014_000000037677.jpg	a woman walking on a path holding an umbrella
COCO_val2014_000000271402.jpg	a man 

COCO_val2014_000000485237.jpg	a large plane is on the runway in the city
COCO_val2014_000000262229.jpg	a group of people playing a game of frisbee
COCO_val2014_000000222094.jpg	a stop sign is in front of a building
COCO_val2014_000000422221.jpg	a man and woman standing next to an elephant
COCO_val2014_000000124135.jpg	a plate with a sandwich and a cup of coffee
COCO_val2014_000000175082.jpg	a group of people sitting around a table eating food
COCO_val2014_000000418696.jpg	a large building with a clock tower on it
COCO_val2014_000000470032.jpg	a bird perched on top of a tree branch
COCO_val2014_000000325664.jpg	a brown bear is sitting on a rock
COCO_val2014_000000535591.jpg	a train traveling down tracks next to a forest
COCO_val2014_000000434060.jpg	a herd of sheep standing on top of a lush green field
COCO_val2014_000000344513.jpg	a train is parked at a train station
COCO_val2014_000000425120.jpg	a group of young men playing soccer on a field
COCO_val2014_000000328780.jpg	a cat is sitt

COCO_val2014_000000519754.jpg	a red refrigerator in a room with a black bag on top of it
COCO_val2014_000000565986.jpg	a man is sitting at a table with a pizza
COCO_val2014_000000359277.jpg	a group of police officers riding motorcycles down a street
COCO_val2014_000000422155.jpg	a couple of people standing next to each other
COCO_val2014_000000446053.jpg	a group of people standing in the snow with skis on
COCO_val2014_000000205636.jpg	a woman laying on her stomach on a bed
COCO_val2014_000000216497.jpg	a kitchen with a large island in the middle of it
COCO_val2014_000000547457.jpg	a woman sitting in a car seat with her dog
COCO_val2014_000000378844.jpg	a train is going down the tracks in the middle of the day
COCO_val2014_000000356043.jpg	a man talking on a cell phone while wearing a hat
COCO_val2014_000000549754.jpg	a man flying through the air while riding a snowboard
COCO_val2014_000000408989.jpg	a woman in a kitchen with a bowl of fruit
COCO_val2014_000000272212.jpg	a couple of cow

...

COCO_val2014_000000449798.jpg	a desk with a laptop , computer monitor , and a keyboard
COCO_val2014_000000185989.jpg	a group of people standing around a table with a microphone
COCO_val2014_000000560108.jpg	a young boy is standing on a skateboard
COCO_val2014_000000010977.jpg	a bathroom with a toilet , sink , and mirror
COCO_val2014_000000113756.jpg	a man riding a skateboard down a street
COCO_val2014_000000221554.jpg	a kitchen with a refrigerator and a counter top
COCO_val2014_000000472228.jpg	a banana and a cup of coffee on a table
COCO_val2014_000000234676.jpg	a man standing on a beach with a surfboard
COCO_val2014_000000388267.jpg	a clock tower is shown in the middle of a building
COCO_val2014_000000033759.jpg	a young boy swinging a baseball bat at a ball
COCO_val2014_000000330050.jpg	two women standing next to each other holding tennis racquets
COCO_val2014_000000311746.jpg	a small purple flower sitting in a green plant
COCO_val2014_000000166255.jpg	a man is riding a horse in a fi

COCO_val2014_000000281429.jpg	a man riding a skateboard down a curvy road
COCO_val2014_000000254745.jpg	a desk with a laptop and a computer monitor sitting on top of it
COCO_val2014_000000464469.jpg	a fire truck parked in front of a fire truck
COCO_val2014_000000010211.jpg	a group of giraffes standing around a fenced in area
COCO_val2014_000000535569.jpg	a pizza and a bowl of salad are on a table
COCO_val2014_000000057535.jpg	a man sitting on a snow covered ski slope
COCO_val2014_000000312020.jpg	a bathroom with a toilet , sink , and a window
COCO_val2014_000000216243.jpg	a baseball player is getting ready to swing the bat
COCO_val2014_000000224542.jpg	a red and white boat sitting on the beach
COCO_val2014_000000406253.jpg	a police officer on a motorcycle driving down the street
COCO_val2014_000000526446.jpg	a baseball player is getting ready to swing at a pitch
COCO_val2014_000000136820.jpg	a woman sitting at a table with a plate of food
COCO_val2014_000000041867.jpg	a bird standing o

COCO_val2014_000000094795.jpg	a dog is standing on a boat in the water
COCO_val2014_000000462006.jpg	a little boy sitting on a bench with a hat
COCO_val2014_000000001205.jpg	a room with a bed , a table and a chair
COCO_val2014_000000259335.jpg	a man carrying a surfboard on top of a sandy beach
COCO_val2014_000000565485.jpg	a train is parked at a train station
COCO_val2014_000000082765.jpg	a bed with a laptop computer sitting on it
COCO_val2014_000000055544.jpg	a kitchen with a sink , refrigerator , oven , and cabinets
COCO_val2014_000000543872.jpg	a refrigerator with a lot of magnets and magnets
COCO_val2014_000000448288.jpg	a group of people standing in the snow with snowboards
COCO_val2014_000000353644.jpg	a man is walking with a dog on a leash
COCO_val2014_000000198724.jpg	a dog is playing with a frisbee in a yard
COCO_val2014_000000121349.jpg	a group of people playing baseball in a field
COCO_val2014_000000113408.jpg	a city street with a lot of buildings and a clock tower
COCO_val2

COCO_val2014_000000261888.jpg	a man riding a bike down a street
COCO_val2014_000000265243.jpg	a street with a lot of traffic on it
COCO_val2014_000000567683.jpg	a group of motorcycles parked in front of a building
COCO_val2014_000000145921.jpg	a large passenger jet sitting on top of an airport tarmac
COCO_val2014_000000325064.jpg	a television is on a table with a lamp and a clock
COCO_val2014_000000553154.jpg	a herd of elephants walking through a field
COCO_val2014_000000181013.jpg	two men playing frisbee in a field of grass
COCO_val2014_000000305609.jpg	a table with a plate of food and a cup of coffee
COCO_val2014_000000015155.jpg	a piece of cake with a fork and a knife on it
COCO_val2014_000000189811.jpg	a pizza with cheese and spinach on a plate
COCO_val2014_000000453755.jpg	a kitchen with a table a refrigerator and a sink
COCO_val2014_000000105867.jpg	a man is eating a donut while sitting at a table
COCO_val2014_000000308576.jpg	a black dog is laying on a blue chair
COCO_val2014_00

COCO_val2014_000000415408.jpg	a red fire hydrant sitting on the sidewalk
COCO_val2014_000000368154.jpg	a stop sign is on a street corner near a street
COCO_val2014_000000559101.jpg	a child sitting on a high chair with a toothbrush in his mouth
COCO_val2014_000000303210.jpg	a man riding a skateboard down a cement ramp
COCO_val2014_000000352587.jpg	a herd of horses standing on top of a dry grass field
COCO_val2014_000000157876.jpg	a plate of food with a sandwich and a drink
COCO_val2014_000000404702.jpg	a group of people sitting on a beach with a lot of food
COCO_val2014_000000581081.jpg	a boat is docked in the water near a city
COCO_val2014_000000550714.jpg	a woman holding a banana with a smile
COCO_val2014_000000441974.jpg	a black and white cat sitting on a bed next to a black cat
COCO_val2014_000000113828.jpg	a double decker bus is parked in front of a building
COCO_val2014_000000446260.jpg	a man wearing a blue shirt and a blue tie
COCO_val2014_000000085298.jpg	a woman holding a tenni

<a id='step4'></a>
## Step 4: Clean up the Captions

In the code cell below, complete the `clean_sentence` function.  It should take a list of integers (corresponding to the variable `output` in **Step 3**) as input and return the corresponding predicted sentence (as a single Python string). 

In [ ]:
# TODO #4: Complete the function.
def clean_sentence(output):
    sentence = ''
    for x in output:
        sentence += ' ' + data_loader.dataset.vocab.idx2word[x]
        sentence = sentence.strip()
    return sentence

After completing the `clean_sentence` function above, run the code cell below.  If the cell returns an assertion error, then please follow the instructions to modify your code before proceeding.

In [ ]:
sentence = clean_sentence(output)
print('example sentence:', sentence)

assert type(sentence)==str, 'Sentence needs to be a Python string!'